In [ ]:
!pip install pytz
!pip install groq -q
!pip install -U langgraph -q
!pip install genai google-colab -q

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_core langgraph
%pip install -qU langchain-google-genai

In [ ]:
import requests
import groq
from langgraph.prebuilt import tools_condition, ToolNode
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph
from langchain_core.tools import tool
from google.colab import userdata
import google.generativeai as genai
import getpass
import os
from datetime import datetime
import pytz

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.checkpoint.memory import MemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

In [ ]:
# Configure the Gemini model
model = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
def extract_key_points(user_input):
    """
    Sends the user's study plan input to the Groq Cloud API using the Llama-3.3-70B model
    to extract key points.

    Parameters:
        user_input (str): The study plan provided by the user.

    Returns:
        list: A list of extracted key points.
    """
    # Define the system prompt and user input for key points extraction
    system_prompt = f"""
    You are an helpful assistant that helps students create study plans. Extract key points from study plans provided by users
    
    """

    # Use the Groq client to send a request
    groq_client = groq.Client(api_key="GROQ_API_KEY")

    try:
        completion = groq_client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": f"Extract key points from the following study plan:\n{user_input}"
                }
            ],
            temperature=0.5,
            max_tokens=512,
            top_p=1,
            stream=False
        )

        # Extract and process the generated content
        generated_content = completion.choices[0].message.content.strip()
        key_points = generated_content.split("\n")

        return key_points

    except Exception as e:
        print(f"Error while communicating with Groq Cloud API: {e}")
        return []




In [ ]:
# Set timezone for Karachi
karachi_tz = pytz.timezone('Asia/Karachi')

# Get current time in Karachi timezone
karachi_time = datetime.now(karachi_tz)
current_date = karachi_time.date()
# Get current day

In [ ]:
def generate_detailed_planning(key_points):
    """
    Generates Mermaid code from the extracted key points.

    Parameters:
        key_points (list): The extracted key points from the study plan.

    Returns:
        str: The Mermaid diagram code.
    """
    # Join key points into a structured prompt
    key_points_text = "\n".join(key_points)

    # Define the system prompt and user input for Mermaid code generation
    system_prompt = (
          f"""
       Create a detailed weekly study plan based on the provided markdown. The plan should focus on the following:

Assignments: List specific tasks (e.g., History Paper, Math Test) with deadlines.
Objectives: Clarify the broader goals of the study plan (e.g., mastering the subject, improving time management).
Support: Provide 1-2 study techniques, focus strategies, or stress management methods to assist in completing assignments and achieving objectives. Only include these techniques when explicitly requested by the user.
Do not break down the tasks day-by-day. Instead, provide a weekly plan that outlines key tasks for the week leading up to deadlines. Ensure the tasks are achievable and aligned with the deadlines and goals. If the user requests, break down further tasks for the next 7 weeks.

Use the provided current date in an f-string format like: f"Today is {current_date} - {karachi_time} - {day}" - . After providing the weekly plan, ask the user if they need a breakdown for the next 7 weeks.
          """
    )

    # Initialize the Groq client
    groq_client = groq.Client(api_key="GROQ_API_KEY")

    try:
        # Send the prompt to the Groq LLM
        completion = groq_client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": f"""
                    Key Points For Planning:\n{key_points_text}

                    Example Output:
Example Output:
Weekly Plan for {current_date} - {karachi_time} - {day}:


Task 1: Review History Paper instructions and outline key points (due by December 17).
Task 2: Begin drafting History Paper (start working on introduction and first section).
Task 3: Study for the Math Test (focus on key formulas and problem-solving).
Task 4: Complete practice problems for Math Test preparation.
Task 5: Start working on the Science Project (gather research materials).
Support:
Study Techniques:
Active recall for studying History and Math.
Pomodoro technique (25 minutes of focused work, 5-minute break) to maintain productivity.
Summary:
This weekly plan ensures the student is focused on high-priority tasks while considering realistic deadlines. It outlines key actions without overwhelming day-to-day granularity, and provides efficient support for accomplishing the tasks.

Ask the user:
Would you like to continue with a detailed breakdown for the next 7 weeks?
                    """
                }
            ],
            temperature=0.5,
            max_tokens=1000,
            top_p=1,
            stream=False
        )

        # Extract the Mermaid code from the response
        generated_mermaid_code = completion.choices[0].message.content.strip()

        # Remove explanations and only keep the code
        start_idx = generated_mermaid_code.find("```")
        end_idx = generated_mermaid_code.rfind("```")
        if start_idx != -1 and end_idx != -1:
            generated_mermaid_code = generated_mermaid_code[start_idx + 3:end_idx].strip()

        # Remove any unwanted "mermaid" keyword
        if generated_mermaid_code.lower().startswith("mermaid"):
            generated_mermaid_code = generated_mermaid_code[len("mermaid"):].strip()

        return generated_mermaid_code

    except Exception as e:
        print(f"Error while communicating with Groq Cloud API for Mermaid code: {e}")
        return ""

In [ ]:

@tool
def intelligent_planning(user_input: str) -> str:
    """Generates intelligent planning based on key points extracted from user input."""
    keypoints = extract_key_points(user_input)
    print(f"Keypoints Generated: {keypoints}")
    detailed_planning = generate_detailed_planning(keypoints)
    return detailed_planning


In [ ]:

api_key = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = api_key

# Method 2: Direct initialization
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=api_key  # Explicitly pass the API key
)

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

tools = [intelligent_planning]

llm_with_tools = llm.bind_tools(tools)

In [ ]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}
    

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[intelligent_planning])

graph_builder.add_node("tools", tool_node)

In [ ]:
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("tools", END)

graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()


In [ ]:
graph = graph_builder.compile(
    checkpointer=memory,
    interrupt_before=["tools"]
)

In [ ]:
user_input = f"""
Struggling to balance my college assignments, soccer practice, and managing my Shopify dropshipping store. 
I've got a marketing project due next week, need to update product listings, and can't miss tomorrow's soccer training.
How can I optimize my time to handle everything without burning out?
"""
config = {"configurable": {"thread_id": "1"}}
events = graph.stream(
    {"messages": [("user", user_input)]},
    config,
    stream_mode="values")


for event in events:
    # if "messages" in event:
        event["messages"][-1].pretty_print()


# snapshot = graph.get_state(config)
# existing_message = snapshot.values["messages"][-1]
# existing_message.pretty_print()


In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass